In [1]:
"""
Created on Thu Oct 25 09:54:12 2018

@author: LongJun
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import config as cfg
import os
import pascal_voc as pascl
import numpy as np
import tensorflow as tf
import network
import datetime
import cv2
from nms import py_cpu_nms
#Val_test class is used to test the output model, the result is mean_ap tested on the pascal_voc 2007 test_imdb
#net： VGG16
#val_data: val_data name
class Val_test(object):   
    def __init__(self, net ,val_data):
        self.net = net
        self.val_data = val_data
        self.overlaps_max = cfg.overlaps_max
        self.overlaps_min = cfg.overlaps_min
        self.ckpt_filename = tf.train.latest_checkpoint(os.path.join(cfg.OUTPUT_DIR))
        self.test_output_dir = cfg.test_output_path
        self.image_output_dir = cfg.image_output_dir
        txtname = os.path.join(self.val_data.devkil_path, self.val_data.name, 'ImageSets', 'Main', self.val_data.phase+'.txt')
        with open(txtname) as f:
            self.image_index = [x.strip() for x in f.readlines()]

    def test_model(self):
        saver = tf.train.Saver()
        _rois_coord = self.net.rois_coord[:,1:5]
        #rois_coord = self.net.rois_coord
        _pred_box = self.net.bbox_pred
        _pred_score = self.net.cls_prob
        #_pred_box_score_arg = tf.argmax(_pred_score, axis=1)
        dect_total_result = [[[] for i in range(self.val_data.num_gtlabels)] for j in range(self.net.num_classes)]
        with tf.Session() as sess:
            saver.restore(sess, self.ckpt_filename)
            for i in range (self.val_data.num_gtlabels):
                print (i, ' image test compeleted')            
                train_data = self.val_data.get()
                image_height = np.array(train_data['image'].shape[1])
                image_width = np.array(train_data['image'].shape[2])
                feed_dict = {self.net.image: train_data['image'], self.net.image_width: image_width,\
                             self.net.image_height: image_height}
                                
                rois_coord, pred_box, pred_score= sess.run([_rois_coord, _pred_box, _pred_score],\
                                                                        feed_dict=feed_dict) 

                for k in range(1, self.net.num_classes):    
                    #pre_class_arg = np.where(pred_score[:,k]>=0)[0]
                    cls_pred_box_target = pred_box[:, k*4:(k+1)*4]
                    cls_pred_box_target = cls_pred_box_target * np.array(cfg.bbox_nor_stdv) + np.array(cfg.bbox_nor_mean)
                    cls_pred_box_coord = self.coord_transform_inv(rois_coord, cls_pred_box_target.astype(np.float32))
                    cls_pred_box_coord = cls_pred_box_coord/train_data['scale'] + 1.0
                    cls_pred_score = pred_score[:, k]
                    #print(cls_pred_box_coord.shape, cls_pred_score.shape)
                    cls_pred_score = cls_pred_score[:, np.newaxis]  
                    cls_pred_target = np.concatenate((cls_pred_box_coord, cls_pred_score), axis=1)
                    keep = py_cpu_nms(cls_pred_target, cfg.test_nms_thresh)
                    cls_pred_target = cls_pred_target[keep, :]
                    dect_total_result[k][i] = cls_pred_target
                   # print (cls_pred_target)
                image_scores = np.hstack([dect_total_result[j][i][:, -1] for j in range(1, self.net.num_classes)]) #
                if len(image_scores) > cfg.test_max_per_image:
                    image_thresh = np.sort(image_scores)[-cfg.test_max_per_image] #
                    for j in range(1, self.net.num_classes):
                        keep = np.where(dect_total_result[j][i][:, -1] >= image_thresh)[0]
                        dect_total_result[j][i] = dect_total_result[j][i][keep, :] #
            mean_ap = self.map_compute(dect_total_result)
            print ('the mean_ap of pascal_voc 2007 is', mean_ap)
        
        
    def coord_transform_inv (self, anchors, boxes):
        anchors = anchors.astype(np.float32)
        anchors = np.reshape(anchors, [-1,4])
        anchor_x = (anchors[:,2] + anchors[:,0]) * 0.5
        anchor_y = (anchors[:,3] + anchors[:,1]) * 0.5
        acnhor_w = (anchors[:,2] - anchors[:,0]) + 1.0
        acnhor_h = (anchors[:,3] - anchors[:,1]) + 1.0
        boxes = np.reshape(boxes, [-1,4])
        boxes_x = boxes[:,0]*acnhor_w + anchor_x
        boxes_y = boxes[:,1]*acnhor_h + anchor_y
        boxes_w = np.exp(boxes[:,2])*acnhor_w
        boxes_h = np.exp(boxes[:,3])*acnhor_h
        coord_x1 = boxes_x - boxes_w*0.5
        coord_y1 = boxes_y - boxes_h*0.5
        coord_x2 = boxes_x + boxes_w*0.5
        coord_y2 = boxes_y + boxes_h*0.5
        coord_result = np.stack([coord_x1, coord_y1, coord_x2, coord_y2], axis=1)
        return coord_result              


#computing map using pascal_voc 2010 algorithm
    def map_compute(self, dect_boxes):
        ap = []
        for cls_ind, cls in enumerate(self.val_data.classes):
            cls_obj = {}
            num_cls_obj = 0
            if cls == 'background':
                 continue
            if not os.path.exists(self.test_output_dir):
                os.mkdir(self.test_output_dir)
            cls_filename = os.path.join(self.test_output_dir, cls+'.txt')
            with open(cls_filename, 'w') as f:
                 for img_ind_dex, image_ind in enumerate(self.image_index):
                      dect_box = dect_boxes[cls_ind][img_ind_dex]
                      if dect_box == []:
                           continue
                      for i in range(dect_box.shape[0]):
                           f.write('{:s} {:2f} {:2f} {:2f} {:2f} {:3f} \n'.format\
                                   (image_ind, dect_box[i][0], dect_box[i][1], dect_box[i][2],\
                                    dect_box[i][3], dect_box[i][4]))
                           
            for gt_label in self.val_data.gt_labels:
                 gt_label_cls_ind = np.where(gt_label['gt_classs']==cls_ind)[0]
                 gt_label_pick_box = gt_label['boxes'][gt_label_cls_ind, :]
                 gt_label_pick_cls = gt_label['gt_classs'][gt_label_cls_ind]
                 diff_pick = gt_label['diff'][gt_label_cls_ind].astype(np.bool)
                 dec_id = [False] * gt_label_cls_ind.size
                 num_cls_obj = num_cls_obj + sum(~diff_pick)
                 cls_obj[gt_label['image_index']] = {'bbox': gt_label_pick_box,\
                                                      'cls': gt_label_pick_cls,\
                                                      'dec_id': dec_id, 'diff': diff_pick}
                 #print (num_cls_obj)
            with open(cls_filename, 'r') as f:
              lines = f.readlines()
            splitlines = [x.strip().split(' ') for x in lines] 
            image_ids = [x[0] for x in splitlines]  
            confidence = np.array([float(x[5]) for x in splitlines])
            BB = np.array([[float(z) for z in x[1:5]] for x in splitlines]) #bounding box                 
            
            nd = len(image_ids)                   
            tp = np.zeros(nd)         
            fp = np.zeros(nd)
            
            if BB.shape[0] > 0:
                 sorted_ind = np.argsort(-confidence) 
                 BB = BB[sorted_ind, :]
                 image_ids = [image_ids[x] for x in sorted_ind] 
                 for d in range(nd): 
                      R = cls_obj[image_ids[d]] 
                      bb = BB[d, :].astype(float) 
                      ovmax = -np.inf 
                      BBGT = R['bbox'].astype(float) 
                      
                      if BBGT.size > 0: 
                           ixmin = np.maximum(BBGT[:, 0], bb[0])
                           iymin = np.maximum(BBGT[:, 1], bb[1])
                           ixmax = np.minimum(BBGT[:, 2], bb[2])
                           iymax = np.minimum(BBGT[:, 3], bb[3])
                           iw = np.maximum(ixmax - ixmin + 1., 0.)
                           ih = np.maximum(iymax - iymin + 1., 0.)
                           inters = iw * ih
                           uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +\
                                  (BBGT[:, 2] - BBGT[:, 0] + 1.) *\
                                  (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)
                           overlaps = inters / uni
                           ovmax = np.max(overlaps) 
                           jmax = np.argmax(overlaps) 
                           
                      if ovmax > cfg.test_fp_tp_thresh:
                           if not R['diff'][jmax]:
                               if not R['dec_id'][jmax]: 
                                    tp[d] = 1.
                                    R['dec_id'][jmax] = 1 
                               else:
                                    fp[d] = 1.
                      else:
                           fp[d] = 1. 
            fp = np.cumsum(fp) 
            tp = np.cumsum(tp) 
            rec = tp / float(num_cls_obj) 
            prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps) 
            ap.append(self.val_data.voc_ap(rec, prec)) 	
            print (np.mean(ap))
        return sum(ap)/(self.net.num_classes - 1.0)
    
                   
    def get_var_list(self, global_variables, ckpt_variables):
        variables_to_restore = []
        for key in global_variables:
            if key.name.split(':')[0] in ckpt_variables:
                variables_to_restore.append(key) 
        return variables_to_restore
    
    

if __name__ == '__main__':
    os.environ['CUDA_VISIBLE_DEVICES'] = cfg.GPU_ID
    net = network.Net(is_training=False)
    val_data = pascl.pascal_voc(cfg.test_imdb_name, 'test', fliped=False)
    test = Val_test(net, val_data)
    print ('start training')
    test.test_model()


/usr/local/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differen

187  image test compeleted
188  image test compeleted
189  image test compeleted
190  image test compeleted
191  image test compeleted
192  image test compeleted
193  image test compeleted
194  image test compeleted
195  image test compeleted
196  image test compeleted
197  image test compeleted
198  image test compeleted
199  image test compeleted
200  image test compeleted
201  image test compeleted
202  image test compeleted
203  image test compeleted
204  image test compeleted
205  image test compeleted
206  image test compeleted
207  image test compeleted
208  image test compeleted
209  image test compeleted
210  image test compeleted
211  image test compeleted
212  image test compeleted
213  image test compeleted
214  image test compeleted
215  image test compeleted
216  image test compeleted
217  image test compeleted
218  image test compeleted
219  image test compeleted
220  image test compeleted
221  image test compeleted
222  image test compeleted
223  image test compeleted
2

491  image test compeleted
492  image test compeleted
493  image test compeleted
494  image test compeleted
495  image test compeleted
496  image test compeleted
497  image test compeleted
498  image test compeleted
499  image test compeleted
500  image test compeleted
501  image test compeleted
502  image test compeleted
503  image test compeleted
504  image test compeleted
505  image test compeleted
506  image test compeleted
507  image test compeleted
508  image test compeleted
509  image test compeleted
510  image test compeleted
511  image test compeleted
512  image test compeleted
513  image test compeleted
514  image test compeleted
515  image test compeleted
516  image test compeleted
517  image test compeleted
518  image test compeleted
519  image test compeleted
520  image test compeleted
521  image test compeleted
522  image test compeleted
523  image test compeleted
524  image test compeleted
525  image test compeleted
526  image test compeleted
527  image test compeleted
5

795  image test compeleted
796  image test compeleted
797  image test compeleted
798  image test compeleted
799  image test compeleted
800  image test compeleted
801  image test compeleted
802  image test compeleted
803  image test compeleted
804  image test compeleted
805  image test compeleted
806  image test compeleted
807  image test compeleted
808  image test compeleted
809  image test compeleted
810  image test compeleted
811  image test compeleted
812  image test compeleted
813  image test compeleted
814  image test compeleted
815  image test compeleted
816  image test compeleted
817  image test compeleted
818  image test compeleted
819  image test compeleted
820  image test compeleted
821  image test compeleted
822  image test compeleted
823  image test compeleted
824  image test compeleted
825  image test compeleted
826  image test compeleted
827  image test compeleted
828  image test compeleted
829  image test compeleted
830  image test compeleted
831  image test compeleted
8

...

4355  image test compeleted
4356  image test compeleted
4357  image test compeleted
4358  image test compeleted
4359  image test compeleted
4360  image test compeleted
4361  image test compeleted
4362  image test compeleted
4363  image test compeleted
4364  image test compeleted
4365  image test compeleted
4366  image test compeleted
4367  image test compeleted
4368  image test compeleted
4369  image test compeleted
4370  image test compeleted
4371  image test compeleted
4372  image test compeleted
4373  image test compeleted
4374  image test compeleted
4375  image test compeleted
4376  image test compeleted
4377  image test compeleted
4378  image test compeleted
4379  image test compeleted
4380  image test compeleted
4381  image test compeleted
4382  image test compeleted
4383  image test compeleted
4384  image test compeleted
4385  image test compeleted
4386  image test compeleted
4387  image test compeleted
4388  image test compeleted
4389  image test compeleted
4390  image test com

4648  image test compeleted
4649  image test compeleted
4650  image test compeleted
4651  image test compeleted
4652  image test compeleted
4653  image test compeleted
4654  image test compeleted
4655  image test compeleted
4656  image test compeleted
4657  image test compeleted
4658  image test compeleted
4659  image test compeleted
4660  image test compeleted
4661  image test compeleted
4662  image test compeleted
4663  image test compeleted
4664  image test compeleted
4665  image test compeleted
4666  image test compeleted
4667  image test compeleted
4668  image test compeleted
4669  image test compeleted
4670  image test compeleted
4671  image test compeleted
4672  image test compeleted
4673  image test compeleted
4674  image test compeleted
4675  image test compeleted
4676  image test compeleted
4677  image test compeleted
4678  image test compeleted
4679  image test compeleted
4680  image test compeleted
4681  image test compeleted
4682  image test compeleted
4683  image test com

4941  image test compeleted
4942  image test compeleted
4943  image test compeleted
4944  image test compeleted
4945  image test compeleted
4946  image test compeleted
4947  image test compeleted
4948  image test compeleted
4949  image test compeleted
4950  image test compeleted
4951  image test compeleted


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:115: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


0.6874316768972664
0.7354234719897195
0.7434722923217191
0.6923488040065384
0.6579735936898059
0.6750189942688579
0.6953374597375855
0.7132330156897515
0.6893911509582482
0.6984662403359347
0.6936201192163525
0.703505353587749
0.7148999916931978
0.7236109162201003
0.7256264184207125
0.706121884259866
0.7087204328302112
0.707153606272569
0.7077586855833486
0.7083405912132499
the mean_ap of pascal_voc 2007 is 0.7083405912132499
